In [ ]:
def run(flow, **parameters):
    """
    Convenience function that figures out a flow's terminal task, then
    runs the flow with the provided parameters and returns that task's result.
    """
    task = list(flow.terminal_tasks())[0]
    state = flow.run(parameters=parameters, return_tasks=[task])
    return state.result[task].result


In [ ]:
from prefect import Flow, Parameter

with Flow('Add one') as flow:
    result = Parameter('x') + 1


In [ ]:
assert run(flow, x=1) == 2
assert run(flow, x=2) == 3
assert run(flow, x=-100) == -99


In [ ]:
with Flow('Add x and y') as flow:
    result = Parameter('x') + Parameter('y')


In [ ]:
assert run(flow, x=1, y=1) == 2
assert run(flow, x=40, y=2) == 42


In [ ]:
from prefect.tasks.control_flow import switch, merge

with Flow('Arithmetic') as flow:
    x, y = Parameter('x'), Parameter('y')
    operations = {
        '+': x + y,
        '-': x - y,
        '*': x * y,
        '/': x / y
    }
    switch(condition=Parameter('op'), cases=operations)
    result = merge(*operations.values())


In [ ]:
assert run(flow, x=1, op='+', y=2) == 3
assert run(flow, x=1, op='-', y=2) == -1
assert run(flow, x=1, op='*', y=2) == 2
assert run(flow, x=1, op='/', y=2) == 0.5


In [ ]:
from prefect import task
from prefect.tasks.control_flow import switch, merge

@task
def parse_input(i):
    x, op, y = i.split(' ')
    return dict(x=float(x), op=op, y=float(y))

with Flow('Arithmetic') as flow:
    inputs = parse_input(Parameter('expression'))
    x, y = inputs['x'], inputs['y']
    operations = {
        '+': x + y,
        '-': x - y,
        '*': x * y,
        '/': x / y
    }
    switch(condition=inputs['op'], cases=operations)
    result = merge(*operations.values())


In [ ]:
assert run(flow, expression='1 + 2') == 3
assert run(flow, expression='1 - 2') == -1
assert run(flow, expression='1 * 2') == 2
assert run(flow, expression='1 / 2') == 0.5


In [ ]:
flow.visualize()
